In [37]:
import pandas as pd
from numpy import nan
import numpy as np

In [3]:
path = "../data/ENDISEG_WEB/conjunto_de_datos_tmodulo_endiseg_web_2022/conjunto_de_datos/conjunto_de_datos_tmodulo_endiseg_web_2022.csv"
df = pd.read_csv(path)
df.head()

,NIP,P4_1,P4_2,P4_3,P4_3A,P4_3B,P4_4,P4_5,P4_6,P4_7,...,P12_5_5,P12_5_6,P12_5_7,P12_6_1,P12_6_2,P12_6_3,P12_6_4,P12_6_5,P12_6_6,ENT
0,1,47,6,NaN,NaN,NaN,2,NaN,2,2,...,1,3,2,2,2,2,2,2,2,11.0
1,2,32,6,NaN,NaN,NaN,2,NaN,2,2,...,1,1,1,1,2,2,2,1,2,15.0
2,3,35,1,1.0,1.0,33.0,2,NaN,2,9,...,1,1,1,2,2,2,2,1,2,23.0
3,4,39,2,1.0,2.0,39.0,2,NaN,2,2,...,1,2,1,2,2,2,2,2,2,9.0
4,5,30,1,1.0,1.0,37.0,2,NaN,2,2,...,1,2,2,2,2,2,2,1,2,7.0


In [32]:
# P8.1: Entonces usted se clasifica...
# P11.6.11 En lo últimos 12 meses ha sido discriminado por su identidad de género y orientacion sexual?
df2 = df[['P8_1', 'P11_6_11', 'ENT']]
df2 = df2[df2['ENT'] != 0] # Eliminamos los datos donde la entidad no ha sido especificada
df2 = df2[df2['ENT'] != nan] # Eliminamos tambien los NA

In [93]:
df3 = df2.groupby(['ENT','P8_1']).size().reset_index()
df3.head()

,ENT,P8_1,0
0,1.0,1,18
1,1.0,2,123
2,1.0,3,107
3,1.0,4,629
4,1.0,5,438


In [94]:
df3['lgbt'] = np.where(df3['P8_1'].isin([1,2,3,6]), 1, 0)

In [95]:
df3 = df3.groupby(['ENT','lgbt']).sum().reset_index().drop('P8_1', axis=1)
df3.head()

,ENT,lgbt,0
0,1.0,0,1067
1,1.0,1,259
2,2.0,0,94
3,2.0,1,176
4,3.0,0,50


In [102]:
df3.iloc[1,2]/df3.iloc[0,2]

0.24273664479850046

In [112]:
df4 = pd.DataFrame({'ent': range(1,33),
    'percentage_lgbt': [df3.iloc[2*i +1,2]/(df3.iloc[2*i +1,2] +df3.iloc[2*i,2]) for i in range(0,32)]})
df4.head()

,ent,percentage_lgbt
0,1,0.195324
1,2,0.651852
2,3,0.618321
3,4,0.450000
4,5,0.590164


In [167]:
df5 = df2.groupby(['ENT','P8_1','P11_6_11']).size().reset_index()
df5 = df5[df5['P8_1'].isin([1,2,3,6])].drop(['P8_1'], axis=1)
df6 = df5.groupby(['ENT','P11_6_11']).sum().reset_index()
df6.columns = ['ent', 'discrimination', 'count']
df6.head()

,ent,discrimination,count
0,1.0,1,11
1,1.0,2,248
2,2.0,1,7
3,2.0,2,169
4,3.0,1,5


In [183]:
df7 = df6.pivot(index = ['ent'], columns=['discrimination'])['count'].reset_index()
df7.columns = ['cve', 'si', 'no']
df7['ratio'] = df7['si']/(df7['si'] + df7['no'])
df7


,cve,si,no,ratio
0,1.0,11.0,248.0,0.042471
1,2.0,7.0,169.0,0.039773
2,3.0,5.0,76.0,0.061728
3,4.0,2.0,34.0,0.055556
4,5.0,6.0,102.0,0.055556
